In [1]:
# Dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from splinter import browser
from selenium import webdriver
#import lxml
#import urllib.request
#import time

## NASA MARS NEWS

In [2]:
url = "https://mars.nasa.gov/news"

In [3]:
# Use selenium webdriver to retrieve page 
driver = webdriver.Firefox()
driver.get(url)
html = driver.page_source

In [4]:
# Scrape the webpage using beautiful soup
soup = bs(html, 'html.parser')

In [5]:
# Latest title and remove the new line 
news_titles = soup.find('div', class_ = 'content_title').get_text(strip=True)
news_titles

'The MarCO Mission Comes to an End'

In [6]:
# Latest Paragraph
news_paragraph = soup.find('div', class_ = 'article_teaser_body').get_text()

In [7]:
news_paragraph

'The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.'

## JPL Mars Space Images - Featured Image

In [8]:
image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

In [9]:
# Retrieve page with the requests module
response = requests.get(image_url)

In [10]:
# Scrape the webpage using beautiful soup
soup = bs(response.text, 'html.parser')

In [14]:
# Featured URL
featured_url = soup.find("article", class_="carousel_item")["style"]
featured_url = featured_url.strip("background-image: url('")
featured_url = featured_url.strip("');")
featured_image_url = "https://www.jpl.nasa.gov" + featured_url

In [15]:
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA18273-1920x1200.jpg'

## Mars Weather

In [16]:
weather_url = "https://twitter.com/marswxreport?lang=en"

In [17]:
# Use selenium webdriver to retrieve page 
response = requests.get(weather_url)

In [18]:
driver = webdriver.Firefox()
driver.get(weather_url)
html = driver.page_source

In [19]:
# Scrape the webpage using beautiful soup
soup = bs(response.text, 'html.parser')

In [20]:
# Extract the tweet text
mars_weather = soup.find(class_='tweet-text').text
mars_weather

'InSight sol 449 (2020-03-01) low -93.6ºC (-136.4ºF) high -11.3ºC (11.7ºF)\nwinds from the SSW at 6.8 m/s (15.1 mph) gusting to 23.9 m/s (53.4 mph)\npressure at 6.30 hPapic.twitter.com/ZcHyd02Y9r'

## Mars Facts

In [21]:
facts_url = "http://space-facts.com/mars/"

In [17]:
mars_table = pd.read_html("http://space-facts.com/mars")

In [18]:
df = mars_table[0]
df.columns = ["Fact","Value"]
# Delete the trailing colon
df["Fact"] = df["Fact"].str[:-1]
df = df.set_index("Fact")
df

In [ ]:
# Convert DataFrame to html 
facts_html_table = df.to_html()
facts_html_table = facts_html_table.replace('\n', '')

## Mars Hemispheres

In [23]:
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [24]:
# Retrieve page with the requests module
response = requests.get(hemisphere_url)

In [25]:
# Scrape the webpage using beautiful soup
soup = bs(response.text, 'html.parser')

In [26]:
hemi_img_url_list = []
title = []

In [27]:
for item in soup.select("a.itemLink"):
    title.append(item.select("h3")[0].get_text())
    url_new = "https://astrogeology.usgs.gov"+item["href"]
    response_new = requests.get(url_new)
    soup_new = bs(response_new.text, 'html.parser')
    hemi_img_url_list.append("https://astrogeology.usgs.gov"+soup_new.select('img.wide-image')[0]['src'])
    

In [28]:
hemisphere_image_url = [{"Title":title[i],'img_url':hemi_img_url_list[i]} for i in range(len(title))]
hemisphere_image_url

[{'Title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'Title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'Title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'Title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]